In [3]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 检查模型和优化器状态文件是否存在
model_path = "model/mnist_net.pt"
optimizer_path = "results/mnist_optimizer.pt"

# 数据预处理
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.286,), (0.353,))  # 假设FashionMNIST数据集已经预处理为[0,1]区间
])

transform_test = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.286,), (0.353,))
])

# 导入dataset
train_set = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform_train)
test_set = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform_test)

# 导入dataloader
train_dataloader = DataLoader(train_set, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=128, shuffle=False)

# 定义CNN模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 第一层卷积
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),  # 卷积层，输入通道1，输出通道16，3x3滤波器
            nn.ReLU(),  # 激活函数
            # nn.MaxPool2d(kernel_size=2, stride=2)  # 最大池化层，2x2窗口
        )
        # 第二层卷积
        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),  # 卷积层，输入通道16，输出通道32
            nn.ReLU(),  # 激活函数
            nn.MaxPool2d(kernel_size=2, stride=2)  # 最大池化层，2x2窗口
        )
        # 全连接层
        self.fc_layer = nn.Sequential(
            nn.Linear(32 * 16 * 16, 256),  # 输入尺寸为32x8x8，输出节点数64
            nn.ReLU(),  # 激活函数
            nn.Dropout(p=0.3),  # 50%的丢弃率
            nn.Linear(256, 10)  # 输出节点数10，对应10个数字类别
        )
    
    def forward(self, x):
        x = self.conv_layer1(x)
        x = self.conv_layer2(x)
        x = x.view(x.size(0), -1) # x = nn.Flatten(x)
        x = self.fc_layer(x)
        return x

# 实例化模型、定义损失函数和优化器
model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

if os.path.exists(model_path) and os.path.exists(optimizer_path):
    model.load_state_dict(torch.load(model_path))
    optimizer.load_state_dict(torch.load(optimizer_path))
    print("模型和优化器状态已加载。")
else:
    print("未找到模型和优化器状态文件，将进行训练。")
    
# 计算宏平均F1分数的辅助函数
def calculate_macro_f1_score(predictions, true_labels, num_classes=10):
    f1_scores = []
    for i in range(num_classes):
        tp = ((predictions == i) & (true_labels == i)).sum().item()
        fp = ((predictions == i) & (true_labels != i)).sum().item()
        fn = ((predictions != i) & (true_labels == i)).sum().item()
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_scores.append(2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0)
    macro_f1_score = np.mean(f1_scores)
    return macro_f1_score

# 训练模型
num_epochs = 10
loss_list = []
accuracy_list = []
macro_f1_score_list = []

def train(num_epochs):
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    os.makedirs(os.path.dirname(optimizer_path), exist_ok=True)
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        correct_preds = 0
        total_samples = 0
        for i, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1) # 预测取最大
            correct_preds += (predicted == labels).sum().item() # 转化标量
            total_samples += labels.size(0)
            if (i+1) % 200 == 0:
                print(f"第{i+1}批, Loss: {loss:.4f}")
        
        epoch_loss /= len(train_dataloader)
        epoch_accuracy = correct_preds / total_samples
        loss_list.append(epoch_loss)
        accuracy_list.append(epoch_accuracy)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}')
        torch.save(model.state_dict(), model_path)
        torch.save(optimizer.state_dict(), optimizer_path)
        
    # 保存模型和优化器状态
    torch.save(model.state_dict(), model_path)
    torch.save(optimizer.state_dict(), optimizer_path)
    
    # 绘制训练过程
    plt.figure(figsize=(10, 8))
    plt.subplot(2, 1, 1)  # 将图表分为2行1列，并定位在第1个子图
    plt.plot(range(1, num_epochs + 1), loss_list, marker='o', linewidth=2, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.xticks(range(1, num_epochs + 1))
    plt.legend()

    plt.subplot(2, 1, 2)  # 定位在第2个子图
    plt.plot(range(1, num_epochs + 1), accuracy_list, marker='o', linewidth=2, label='Training Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.xticks(range(1, num_epochs + 1))
    plt.legend()

    plt.tight_layout()
    plt.show()


# 评估模型
def evaluate(model, test_dataloader):
    model.eval()
    correct_preds = 0
    total_samples = 0
    total_preds = []
    total_labels = []
    
    with torch.no_grad():
        for images, labels in test_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_preds.extend(predicted.view(-1).cpu().numpy())
            total_labels.extend(labels.view(-1).cpu().numpy())
            
            correct_preds += (predicted == labels).sum().item()
            total_samples += labels.size(0)
    
    accuracy = correct_preds / total_samples
    macro_f1_score = calculate_macro_f1_score(np.array(total_preds), np.array(total_labels))
    
    print(f'Test Accuracy: {accuracy:.4f}')
    print(f'Test Macro F1 Score: {macro_f1_score:.4f}')

classes = ["T-shirt/top", "Trouser", "pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle"]
if __name__ == "__main__":
    if os.path.exists(model_path) and os.path.exists(optimizer_path):
        evaluate(model, test_dataloader)
        model.eval()
        with torch.no_grad():
            for images, labels in test_dataloader:
                images, labels = images.to(device), labels.to(device)
                output = model(images)
                pre = torch.argmax(output, dim=1)
                
                # 遍历批次中的每个元素
                for pred, label in zip(pre, labels):
                    result = pred.item()
                    label = label.item()
                    print(f"预测值{classes[result]},真实值{classes[label]}")
                
    else:
        train(num_epochs)


模型和优化器状态已加载。
Test Accuracy: 0.9268
Test Macro F1 Score: 0.9268
预测值Ankle,真实值Ankle
预测值pullover,真实值pullover
预测值Trouser,真实值Trouser
预测值Trouser,真实值Trouser
预测值Shirt,真实值Shirt
预测值Trouser,真实值Trouser
预测值Coat,真实值Coat
预测值Shirt,真实值Shirt
预测值Sandal,真实值Sandal
预测值Sneaker,真实值Sneaker
预测值Coat,真实值Coat
预测值Sandal,真实值Sandal
预测值Sneaker,真实值Sneaker
预测值Dress,真实值Dress
预测值Coat,真实值Coat
预测值Trouser,真实值Trouser
预测值pullover,真实值pullover
预测值Shirt,真实值Coat
预测值Bag,真实值Bag
预测值T-shirt/top,真实值T-shirt/top
预测值pullover,真实值pullover
预测值Sandal,真实值Sandal
预测值Sneaker,真实值Sneaker
预测值Ankle,真实值Ankle
预测值Trouser,真实值Trouser
预测值pullover,真实值Coat
预测值Shirt,真实值Shirt
预测值T-shirt/top,真实值T-shirt/top
预测值Ankle,真实值Ankle
预测值Dress,真实值Dress
预测值Bag,真实值Bag
预测值Bag,真实值Bag
预测值Dress,真实值Dress
预测值Dress,真实值Dress
预测值Bag,真实值Bag
预测值T-shirt/top,真实值T-shirt/top
预测值Sneaker,真实值Sneaker
预测值Sandal,真实值Sandal
预测值Sneaker,真实值Sneaker
预测值Ankle,真实值Ankle
预测值T-shirt/top,真实值Shirt
预测值Trouser,真实值Trouser
预测值Shirt,真实值Dress
预测值Sneaker,真实值Sneaker
预测值Shirt,真实值Shirt
预测值Sneaker,真实值Sneaker
预测值pullove